# Formateo de Datos (Tidy Data)
Como mencionamos anteriormente. Nos aprovecharemos de la forma de la base de datos para darle un formato más adecuado para su análisis. La estructura constará de usar un múlti indice en la parte de las columnas y de renglonres los municipios. 

## Extracción de Datos crudos (Raw Data)

In [37]:
import pandas as pd
import numpy as np

Sonora = pd.read_excel("..\\medium_data\\book_son.xlsx", sheet_name=None)
Municipios = list(Sonora.keys())

In [63]:
def transform(mun: str):
    df = Sonora[mun].copy()
    df.drop(columns=["Unnamed: 0.1","Unnamed: 0"], inplace=True)
    df.dropna(how = 'all', axis = 0, inplace=True)
    df.reset_index(drop=True,inplace=True)

    abs_index = []
    for i in range(len(df)):
        if df.iloc[i, 1] == 'Absoluto':
            abs_index.append(i)
    abs_index.append(len(df))
    new_dfs = []
    for a in range(len(abs_index) - 1):
        u = df.iloc[abs_index[a]:abs_index[a + 1],:]
        u = u.dropna(how = 'all', axis = 1) 
        new_dfs.append(u)
    return new_dfs


data_to_tidy ={}
for mun in Municipios:
    nl = []
    y = transform(mun)
    for df in y:
        df.columns = df.iloc[0,:]
        df = df.iloc[1:,:]
        df.reset_index(drop=True, inplace=True)
        nl.append(df)
    data_to_tidy[mun] = nl


 # Preliminares

In [64]:
import regex as re

def footnote(data: pd.DataFrame):
    extra_information = []
    extra_codes = ['Nota','^\d']
    remove_codes = ['Fuente']
    for i in list(data.iloc[:,0]):
        for code in extra_codes:
            if re.findall(code, i):
                extra_information.append(i)
                data = data.drop(index = i)
        for code in remove_codes:
            if re.findall(code, i):
                data = data.drop(index = list(data.iloc[:,0]).index(i))
    return data, extra_information


descriptor = pd.read_excel("..\\raw_data\\descriptor.xlsx")

descriptor['Origen'] = descriptor['Origen'].ffill()
descriptor.set_index(['Origen','Némonico'], drop=True, inplace=True)
sim_cats = descriptor.index.get_level_values(0).unique()

def filter_mi(cat):
    return descriptor.loc[(sim_cats[cat],slice(None)),:]

mun_0 = Municipios[0]

# Composición Territorial

In [65]:
test_to_tidy = data_to_tidy[mun_0]

ct  = test_to_tidy[0]
ct, _ = footnote(ct)

ct

2,Composición territorial,Absoluto,Porcentaje
0,Localidades,25,-
1,Urbanas,1,4
2,Rurales,24,96
3,AGEB,14,-
4,Urbanas,11,78.6
5,Rurales,3,21.4
6,Manzanas,223,-
7,Urbanas,135,60.5
8,Rurales,88,39.5


In [66]:
def abspercols(data):
    y = []
    for d in data:
        y += [d + 'A', d + '%']
    return y

cols = filter_mi(0).index.get_level_values(1)
cols 

comter = pd.DataFrame(index = Municipios, columns = abspercols(cols))
ap_01 = []
for i in range(len(ct)):
    ap_01 += list(ct.iloc[i,1:])
 
comter.iloc[0,:] = ap_01
comter

,LOCTOTA,LOCTOT%,LOCURBA,LOCURB%,LOCRURA,LOCRUR%,AGBTOTA,AGBTOT%,AGBURBA,AGBURB%,AGBRURA,AGBRUR%,MANTOTA,MANTOT%,MANURBA,MANURB%,MANRURA,MANRUR%
Aconchi,25,-,1,4,24,96,14,-,11,78.6,3,21.4,223,-,135,60.5,88,39.5
Agua Prieta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Alamos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Altar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Arivechi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Villa Pesqueira,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Yécora,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Gral. Plutarco Elías Calles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Benito Juárez,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
def fill_mun(cat, mun):
    test_to_tidy = data_to_tidy[mun]
    ct  = test_to_tidy[cat]
    ct, _ = footnote(ct)
    ap_01 = []
    for i in range(len(ct)):
        ap_01 += list(ct.iloc[i,1:])
    return ap_01

def fill_df(cat):
    cols = filter_mi(cat).index.get_level_values(1)
    comter = pd.DataFrame(index = Municipios, columns = abspercols(cols))
    for m in Municipios:
        comter.loc[mun,:] = fill_mun(cat, m)


# 

In [70]:
ug = test_to_tidy[1]
ug, _= footnote(ug)
ug



for m, mun in enumerate(Municipios):
    geo_ubication(m, mun, geo_son)
geo_son

13,Ubicación geográfica,Absoluto,Porcentaje
0,Coordenadas,Entre los paralelos 29° 41’ y 29° 53’ de latit...,-
1,Altitud,Entre 500 y 2 200 metros.,-
2,Colindancias,Colinda al norte con los municipios de San Fel...,-
3,Superficie (Km2),368.2,0.2
4,Densidad de población (habitantes/km2),7,-
